In [1]:
%pip install neo4j pandas

  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl (11.4 MB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "neoneoneo"


In [3]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USER, NEO4J_PASSWORD)
)

driver.verify_connectivity()


In [3]:
with driver.session() as session:
    def get_answer(tx):
        result = tx.run("RETURN 42 AS answer")
        for record in result:
            print(record["answer"])

        summary = result.consume()
        print("available after", summary.result_available_after, "ms and consumed after", summary.result_consumed_after, "ms")
    
        return result

    result = session.execute_read(get_answer)
    print(type(result))


result

42
available after 22 ms and consumed after 0 ms
<class 'neo4j._sync.work.result.Result'>


In [33]:
from datetime import datetime

def send_to_ui(result):
    for record in result:
        print(datetime.now(), dict(record))

driver.execute_query(
    "UNWIND range(1, 10) AS r RETURN r",
    result_transformer_=send_to_ui
)

2025-02-19 10:38:11.263694 {'r': 1}
2025-02-19 10:38:11.264013 {'r': 2}
2025-02-19 10:38:11.264071 {'r': 3}
2025-02-19 10:38:11.264108 {'r': 4}
2025-02-19 10:38:11.264141 {'r': 5}
2025-02-19 10:38:11.264227 {'r': 6}
2025-02-19 10:38:11.264279 {'r': 7}
2025-02-19 10:38:11.264312 {'r': 8}
2025-02-19 10:38:11.264344 {'r': 9}
2025-02-19 10:38:11.264374 {'r': 10}


In [34]:
res = driver.execute_query(
    "UNWIND range(1, 10) AS r RETURN r"
)
type(res)

neo4j._work.eager_result.EagerResult

In [35]:
from datetime import datetime

def send_to_ui(result):
    for record in result:
        print(datetime.now(), dict(record))

driver.execute_query(
    "UNWIND range(1, 10) AS r RETURN r",
    result_transformer_=send_to_ui
)

2025-02-19 14:51:41.420680 {'r': 1}
2025-02-19 14:51:41.420835 {'r': 2}
2025-02-19 14:51:41.420864 {'r': 3}
2025-02-19 14:51:41.420888 {'r': 4}
2025-02-19 14:51:41.421151 {'r': 5}
2025-02-19 14:51:41.421194 {'r': 6}
2025-02-19 14:51:41.421219 {'r': 7}
2025-02-19 14:51:41.421240 {'r': 8}
2025-02-19 14:51:41.421260 {'r': 9}
2025-02-19 14:51:41.421279 {'r': 10}


In [3]:
from neo4j.time import DateTime, Date, Time, LocalTime, LocalDateTime, Duration

time = LocalTime(12, 30, 0)
print(time)


ImportError: cannot import name 'LocalTime' from 'neo4j.time' (/Users/adam/graphacademy/courses/.venv/lib/python3.12/site-packages/neo4j/time/__init__.py)

In [43]:
year = 2012
month = 3
day = 4

d = Date(year, month, day)


d.to_iso_format()


AttributeError: Date has no attribute 'to_iso_format'

In [47]:
date = Date.today()

print(date.to_ordinal())
print(date.to_native())
# print(date.to_clock_time(20))
print(date.to_iso_format())

739301
2025-02-19


AttributeError: Date has no attribute 'to_iso_format'

In [48]:

year = 2018
month = 4
day = 30

date = Date(year, month, day)

print(date.to_ordinal())
print(date.to_native())
print(date.to_clock_time())
print(date.to_iso_format())

736814
2018-04-30


TypeError: Date.to_clock_time() missing 1 required positional argument: 'epoch'

In [3]:
from datetime import timezone, timedelta
from neo4j.time import DateTime


DateTime(
    2024, 5, 15, 14, 30, 0, 
    tzinfo=timezone(timedelta(hours=2))
  )

neo4j.time.DateTime(2024, 5, 15, 14, 30, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))

In [4]:
from neo4j.spatial import Point, CartesianPoint, WGS84Point 

point = CartesianPoint(1, 2)

point.x
point.y
point.z


TypeError: Point.__new__() takes 2 positional arguments but 3 were given

In [22]:
WGS84Point((1, 2)).latitude


2.0

In [36]:
records, _, _ = driver.execute_query(""" 
RETURN point({x: 1, y: 2}) AS cartesian, 
                                     point({x: 1, y: 2, z: 3}) AS cartesian3d,
        point({latitude: 1, longitude: 2}) AS wgs84,
        point({latitude: 1, longitude: 2, z: 3}) AS wgs843d
""")

(records[0]['cartesian'], records[0]['wgs84'], records[0]['cartesian3d'], records[0]['wgs843d'])


(POINT(1.0 2.0), POINT(2.0 1.0), POINT(1.0 2.0 3.0), POINT(2.0 1.0 3.0))

In [26]:
(type(records[0]['cartesian']), type(records[0]['wgs84']), type(records[0]['cartesian3d']), type(records[0]['wgs843d']))


(neo4j._spatial.CartesianPoint,
 neo4j._spatial.WGS84Point,
 neo4j._spatial.CartesianPoint,
 neo4j._spatial.WGS84Point)

In [40]:
records[0]['wgs843d'].x

2.0

In [30]:
records, _, _ = driver.execute_query(""" 
WITH point({x: 1, y: 2}) AS cartesian,
        point({latitude: 1, longitude: 2}) AS wgs84,
        point({latitude: 1, longitude: 2, height: 100}) AS wgs843d
RETURN cartesian, wgs84, wgs843d, point.distance(cartesian, cartesian) AS distance
""")

records[0]


<Record cartesian=POINT(1.0 2.0) wgs84=POINT(2.0 1.0) wgs843d=POINT(2.0 1.0 100.0) distance=0.0>

In [20]:
x, y = records[0]['cartesian']

x, y, records[0]['cartesian'].srid, records[0]['cartesian'].x, records[0]['cartesian'].latitude

AttributeError: 'CartesianPoint' object has no attribute 'latitude'

In [21]:
wsg = records[0]['wgs84']
wsg.x, wsg.y, wsg.srid, wsg.longitude, wsg.latitude


(2.0, 1.0, 4326, 2.0, 1.0)

In [32]:
wsg = records[0]['wgs843d']

longitude, latitude, height = wsg

longitude, latitude, height

(2.0, 1.0, 100.0)

In [13]:
from neo4j.spatial import Point, CartesianPoint, WGS84Point 

(records[0]['cartesian'].srid, records[0]['wgs84'].srid, records[0]['cartesian3d'].srid, records[0]['wgs843d'].srid)


KeyError: 'cartesian3d'

In [15]:
records[0]['wgs84'].x


1.0

In [32]:
records, _, _ = driver.execute_query("""
RETURN point({latitude: 1, longitude: 1}).latitude AS lat
""")

type(records[0]['lat'])


float

## Exceptions

In [4]:
driver.execute_query("CREATE CONSTRAINT FOR (n:Person) REQUIRE n.name IS UNIQUE")


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x12215c680>, keys=[])

In [9]:
from neo4j.exceptions import Neo4jError

try: 
    driver.execute_query("CREATE (p:Person {name: 'John'})")
except Exception as e:
    print(type(e))
    print(e.code)
    print(e.message)
    print(e.gql_status)
    # print(e.gql_description)
    # print(e)



<class 'neo4j.exceptions.ConstraintError'>
Neo.ClientError.Schema.ConstraintValidationFailed
Node(3066) already exists with label `Person` and property `name` = 'John'
50N42
